**Team Visualization**

Goal: For all 30 teams, compare the *expected* number of runs scored per game calculated using the Markov Chain to the *actual* number of runs scored per game in 2016.

In [1]:
from markov_functions import (find_team_atbats, import_raw_batting_data, run_matrix,
                            find_states, make_empty_transition_matrix, 
                            make_transition_matrix, team_markov,
                            team_markov_from_raw)

batting=import_raw_batting_data(verbose = False)

In [2]:
# Show the expected runs for 9 innings of the Minnesota Twins
print(team_markov_from_raw(batting, "MIN"))

5.81307069903


In [3]:
# Now find the expected runs for 9 innings of all 30 teams
# (This takes some time to run...)
team_codes = batting.visteam.unique()
team_runs_per_9 = {team:team_markov_from_raw(batting, team) for team in team_codes}
print(team_runs_per_9)

{'CIN': 5.7835963264503105, 'WAS': 5.9850904772612061, 'NYN': 5.3796663787204437, 'PHI': 5.1206321094137026, 'DET': 6.136764025401769, 'CHA': 5.2560025844894458, 'OAK': 4.7050994570590925, 'PIT': 5.5507411362189272, 'TEX': 6.0551656758336572, 'SDN': 5.1140017257752479, 'BAL': 5.8845932407912587, 'BOS': 6.6382131279814907, 'ARI': 5.9292154301695366, 'SLN': 5.8278414600815198, 'TOR': 5.5759519780219282, 'HOU': 5.9071522836238488, 'ATL': 5.0923221120316331, 'MIN': 5.8130706990279499, 'SEA': 5.7671807993032171, 'COL': 6.4761877810536195, 'TBA': 5.5336832918836532, 'MIA': 5.1848046917608146, 'CLE': 6.2236148023533939, 'ANA': 5.2064281316535288, 'KCA': 5.0977499601756042, 'SFN': 5.5146144643686412, 'LAN': 5.4445949177522834, 'NYA': 5.1306437446854058, 'CHN': 6.4739706857570125, 'MIL': 5.9167171091630042}


How do these predicted runs-per-9-innings compare to the actual 2016 runs per game? We'll compare the two using data from http://www.baseball-reference.com/leagues/MLB/2016.shtml. Unfortunately, our data set uses a set of team codes that is slightly different from this data, so we'll re-code the index of our imported data frame accordingly.

In [4]:
import pandas as pd
team_2016_data = pd.read_csv('team_2016_stats.csv', index_col = 'team_name')
# Data from http://www.baseball-reference.com/leagues/MLB/2016.shtml

code_exchange = {'ARI':'ARI', 'ATL':'ATL', 'BAL':'BAL', 'BOS':'BOS', 'CHC':'CHN', 'CHW':'CHA', 
                 'CIN':'CIN', 'CLE':'CLE', 'COL':'COL', 'DET':'DET', 'HOU':'HOU', 'KCR':'KCA', 
                 'LAA':'ANA', 'LAD':'LAN', 'MIA':'MIA', 'MIL':'MIL', 'MIN':'MIN', 'NYM':'NYN', 
                 'NYY':'NYA', 'OAK':'OAK', 'PHI':'PHI', 'PIT':'PIT', 'SDP':'SDN', 'SEA':'SEA', 
                 'SFG':'SFN', 'STL':'SLN', 'TBR':'TBA', 'TEX':'TEX', 'TOR':'TOR', 'WSN':'WAS'}

team_2016_data.index = [code_exchange[i] for i in team_2016_data.index]

print(team_2016_data['runs_per_game'])

ARI    4.64
ATL    4.03
BAL    4.59
BOS    5.42
CHN    4.99
CHA    4.23
CIN    4.42
CLE    4.83
COL    5.22
DET    4.66
HOU    4.47
KCA    4.17
ANA    4.43
LAN    4.48
MIA    4.07
MIL    4.14
MIN    4.46
NYN    4.14
NYA    4.20
OAK    4.03
PHI    3.77
PIT    4.50
SDN    4.23
SEA    4.74
SFN    4.41
SLN    4.81
TBA    4.15
TEX    4.72
TOR    4.69
WAS    4.71
Name: runs_per_game, dtype: float64


In [5]:
from bokeh.io import output_notebook, show, vform, curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import Range1d
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import Select
from bokeh.models.annotations import Label
from bokeh.layouts import column, row
output_notebook()

Loading BokehJS ...

In [20]:
# create a new plot with default tools, using figure

team_league = ['AL', 'AL', 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'AL', 'AL', 'AL',
 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'NL', 'NL', 'AL', 'NL', 'NL',
 'NL', 'NL', 'NL', 'NL', 'NL', 'NL']

expected_runs = [team_runs_per_9[i] for i in team_codes]
#print(exp_runs)

actual_runs = [team_2016_data.at[i,'runs_per_game'] for i in team_codes]
#print(actual_runs)

source = ColumnDataSource(
        data={
            "expected_runs":expected_runs,
            "actual_runs":actual_runs,
            "team":team_codes,
            "league":team_league
        }
    )

hover = HoverTool(
        tooltips=[
            ("Team", "@team"),
            ("Expected Runs", "@expected_runs"),
            ("Actual Runs", "@actual_runs"),
            ("League", "@league")
        ]
    )


p = figure(plot_width=600, plot_height=600, tools=[hover], title="This Markov Chain Prediction is Quite Awful!")
p.x_range = Range1d(3, 7)
p.y_range = Range1d(3, 7)

p.xaxis.axis_label = "Expected Runs Per Game"
p.yaxis.axis_label = "Actual Runs Per Game"


p.circle(x="expected_runs", y="actual_runs", size=15, line_color="black", fill_color="firebrick", 
         fill_alpha=0.5, source=source)
p.line(x=[3,9],y=[3,9], line_width=4, line_color="navy", alpha=0.6)
p.line(x=[3,9],y=[3,9], line_width=150, line_color="navy", alpha=0.1)

p.add_layout(Label(x=6.638, y=5.42, x_offset=-17, y_offset=10, text="BOS", text_color = "black"))
p.add_layout(Label(x=5.121, y=3.77, x_offset=-15, y_offset=-30, text="PHI", text_color = "black"))
p.add_layout(Label(x=4.705, y=4.030, x_offset=-17, y_offset=-30, text="OAK", text_color = "black"))
p.add_layout(Label(x=5.917, y=4.14, x_offset=-12, y_offset=-30, text="MIL", text_color = "black"))


show(p)

In [7]:
df = pd.DataFrame(index=team_codes)
df['expected_runs'] = expected_runs
df['actual_runs'] = [team_2016_data.at[i,'runs_per_game'] for i in team_codes]
df['league'] = [ 'AL', 'AL', 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'AL', 'AL', 'AL',
                 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'NL', 'NL', 'AL', 'NL', 'NL',
                 'NL', 'NL', 'NL', 'NL', 'NL', 'NL']
#df.index = df['team_code']
print(df)

     expected_runs  actual_runs league
BAL       5.884593         4.59     AL
TOR       5.575952         4.69     AL
TBA       5.533683         4.15     AL
ATL       5.092322         4.03     NL
NYA       5.130644         4.20     AL
OAK       4.705099         4.03     AL
HOU       5.907152         4.47     NL
CLE       6.223615         4.83     AL
COL       6.476188         5.22     NL
SEA       5.767181         4.74     AL
CHA       5.256003         4.23     AL
ANA       5.206428         4.43     AL
TEX       6.055166         4.72     AL
SFN       5.514614         4.41     NL
MIN       5.813071         4.46     AL
DET       6.136764         4.66     AL
ARI       5.929215         4.64     NL
KCA       5.097750         4.17     AL
SDN       5.114002         4.23     NL
SLN       5.827841         4.81     NL
PIT       5.550741         4.50     NL
BOS       6.638213         5.42     AL
MIL       5.916717         4.14     NL
WAS       5.985090         4.71     NL
PHI       5.120632       

In [15]:
# create a new plot with default tools, using figure

team_league = ['AL', 'AL', 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'AL', 'AL', 'AL',
 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'NL', 'NL', 'AL', 'NL', 'NL',
 'NL', 'NL', 'NL', 'NL', 'NL', 'NL']

expected_runs = [team_runs_per_9[i] for i in team_codes]
#print(exp_runs)

actual_runs = [team_2016_data.at[i,'runs_per_game'] for i in team_codes]
#print(actual_runs)

#source = ColumnDataSource(
#        data=dict(
#            expected_runs=expected_runs,
#            actual_runs=actual_runs,
#            team=team_codes,
#            league=team_league
#        )
#    )

hover = HoverTool(
        tooltips=[
            ("Team", "@team"),
            ("Expected Runs", "@expected_runs"),
            ("Actual Runs", "@actual_runs"),
            ("League", "@league")
        ]
    )


def get_dataset(league_option):
    df = dict(
            expected_runs=expected_runs,
            actual_runs=actual_runs,
            team=team_codes,
            league=team_league
        )
    return ColumnDataSource(data=df)



def make_plot(source, title_text):
    p = figure(plot_width=600, plot_height=600, tools=[hover], title = title_text)
    p.x_range = Range1d(3, 7)
    p.y_range = Range1d(3, 7)

    p.xaxis.axis_label = "Expected Runs Per Game"
    p.yaxis.axis_label = "Actual Runs Per Game"


    p.circle(x=expected_runs, y=actual_runs, size=15, line_color="black", fill_color="firebrick", 
             fill_alpha=0.5, source=source)
    p.line(x=[3,9],y=[3,9], line_width=4, line_color="navy", alpha=0.6)
    p.line(x=[3,9],y=[3,9], line_width=150, line_color="navy", alpha=0.1)

    p.add_layout(Label(x=6.638, y=5.42, x_offset=-17, y_offset=10, text="BOS", text_color = "black"))
    p.add_layout(Label(x=5.121, y=3.77, x_offset=-15, y_offset=-30, text="PHI", text_color = "black"))
    p.add_layout(Label(x=4.705, y=4.030, x_offset=-17, y_offset=-30, text="OAK", text_color = "black"))
    p.add_layout(Label(x=5.917, y=4.14, x_offset=-12, y_offset=-30, text="MIL", text_color = "black"))

    return p # show the results

def update_plot(attrname, old, new):
    print("hello world")
    city = city_select.value
    plot.title.text = "Weather data for " + cities[city]['title']

    src = get_dataset(df, cities[city]['airport'], distribution_select.value)
    source.data.update(src.data)

    

output_notebook()
league = 'All Teams'
league_options = ['All Teams', 'AL', 'NL']

league_select = Select(value=league, title='League', options=league_options)
league_select.on_change('value', update_plot) 

source = get_dataset("AL")
plot = make_plot(source, "This is a Markov Chain Prediction")

controls = column(league_select)

curdoc().add_root(row(plot, controls))
curdoc().title = "Markov Chain Plot"

Loading BokehJS ...

In [16]:
from os.path import join, dirname
import datetime

import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure

STATISTICS = ['record_min_temp', 'actual_min_temp', 'average_min_temp', 'average_max_temp', 'actual_max_temp', 'record_max_temp']

def get_dataset(src, name, distribution):
    df = src[src.airport == name].copy()
    del df['airport']
    df['date'] = pd.to_datetime(df.date)
    # timedelta here instead of pd.DateOffset to avoid pandas bug < 0.18 (Pandas issue #11925)
    df['left'] = df.date - datetime.timedelta(days=0.5)
    df['right'] = df.date + datetime.timedelta(days=0.5)
    df = df.set_index(['date'])
    df.sort_index(inplace=True)
    if distribution == 'Smoothed':
        window, order = 51, 3
        for key in STATISTICS:
            df[key] = savgol_filter(df[key], window, order)

    return ColumnDataSource(data=df)

def make_plot(source, title):
    plot = figure(x_axis_type="datetime", plot_width=800, tools="", toolbar_location=None)
    plot.title.text = title

    plot.quad(top='record_max_temp', bottom='record_min_temp', left='left', right='right',
              color=Blues4[2], source=source, legend="Record")
    plot.quad(top='average_max_temp', bottom='average_min_temp', left='left', right='right',
              color=Blues4[1], source=source, legend="Average")
    plot.quad(top='actual_max_temp', bottom='actual_min_temp', left='left', right='right',
              color=Blues4[0], alpha=0.5, line_color="black", source=source, legend="Actual")

    # fixed attributes
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Temperature (F)"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    return plot

def update_plot(attrname, old, new):
    city = city_select.value
    plot.title.text = "Weather data for " + cities[city]['title']

    src = get_dataset(df, cities[city]['airport'], distribution_select.value)
    source.data.update(src.data)

city = 'Austin'
distribution = 'Discrete'

cities = {
    'Austin': {
        'airport': 'AUS',
        'title': 'Austin, TX',
    },
    'Boston': {
        'airport': 'BOS',
        'title': 'Boston, MA',
    },
    'Seattle': {
        'airport': 'SEA',
        'title': 'Seattle, WA',
    }
}

city_select = Select(value=city, title='City', options=sorted(cities.keys()))
distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])

df = pd.read_csv('2015_weather.csv')
source = get_dataset(df, cities[city]['airport'], distribution)
plot = make_plot(source, "Weather data for " + cities[city]['title'])

city_select.on_change('value', update_plot)
distribution_select.on_change('value', update_plot)

controls = column(city_select, distribution_select)

curdoc().add_root(row(plot, controls))
curdoc().title = "Weather"

In [14]:
df = pd.DataFrame(index=team_codes)
df['expected_runs'] = expected_runs
df['actual_runs'] = [team_2016_data.at[i,'runs_per_game'] for i in team_codes]
df['league'] = [ 'AL', 'AL', 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'AL', 'AL', 'AL',
                 'AL', 'NL', 'AL', 'AL', 'NL', 'AL', 'NL', 'NL', 'NL', 'AL', 'NL', 'NL',
                 'NL', 'NL', 'NL', 'NL', 'NL', 'NL']
#df.index = df['team_code']
print(df)
df.to_csv("team_visualization/team_data.csv")

     expected_runs  actual_runs league
BAL       5.884593         4.59     AL
TOR       5.575952         4.69     AL
TBA       5.533683         4.15     AL
ATL       5.092322         4.03     NL
NYA       5.130644         4.20     AL
OAK       4.705099         4.03     AL
HOU       5.907152         4.47     NL
CLE       6.223615         4.83     AL
COL       6.476188         5.22     NL
SEA       5.767181         4.74     AL
CHA       5.256003         4.23     AL
ANA       5.206428         4.43     AL
TEX       6.055166         4.72     AL
SFN       5.514614         4.41     NL
MIN       5.813071         4.46     AL
DET       6.136764         4.66     AL
ARI       5.929215         4.64     NL
KCA       5.097750         4.17     AL
SDN       5.114002         4.23     NL
SLN       5.827841         4.81     NL
PIT       5.550741         4.50     NL
BOS       6.638213         5.42     AL
MIL       5.916717         4.14     NL
WAS       5.985090         4.71     NL
PHI       5.120632       

It's clear from the graph that our model consistently **overestimates** the number of runs per game. Likely this is due to an error in the transition matrices that ignores non-transitions like pitching changes, etc. On the other hand, the shape looks pretty linear, which is good.